In [29]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.optimize as opt
from lmfit import models
from lmfit import Model

In [5]:
pc_henrik = '/home/arrow13/Git/FP/Roentgen/'
pc_frederik = '/home/frederik/Dokumente/FP/Michelson/'

In [6]:
var_pc = pc_frederik

In [9]:
cd $var_pc/Messdaten

/home/frederik/Dokumente/FP/Michelson/Messdaten


# Spannung in Abhängigkeit der Chopperfrequenz

In [47]:
data_chopper = np.genfromtxt('chopper.dat',delimiter='\t').T

In [48]:
def chopper(x,A,B):
    return np.exp(A*np.log(x)+B)

In [49]:
model_chopper = Model(chopper,prefix='chop')
guess_chopper = model_chopper.make_params()
guess_chopper['chopA'].set(-0.8)
guess_chopper['chopB'].set(5)
out_chopper = model_chopper.fit(data_chopper[1],guess_chopper,x=data_chopper[0],weights=1/data_chopper[3])

In [50]:
out_chopper.plot_fit()
plt.show()

In [51]:
print(out_chopper.fit_report())

[[Model]]
    Model(chopper, prefix='chop')
[[Fit Statistics]]
    # function evals   = 36
    # data points      = 11
    # variables        = 2
    chi-square         = 7.808
    reduced chi-square = 0.868
[[Variables]]
    chopB:   8.91330758 +/- 0.096268 (1.08%) (init= 5)
    chopA:  -1.03784582 +/- 0.034072 (3.28%) (init=-0.8)
[[Correlations]] (unreported correlations are <  0.100)
    C(chopB, chopA)              = -0.982 


In [53]:
plt.errorbar(data_chopper[0],data_chopper[1],xerr=data_chopper[2],yerr=data_chopper[3],fmt='. ',label="Messdaten")
plt.plot(np.arange(5,60,0.1),out_chopper.eval(x=np.arange(5,60,0.1)),label='Fit')
plt.xlabel('Chopperfrequenz / Hz', fontsize=30)
plt.ylabel('Spannung /V', fontsize=30)
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)
plt.xscale('log')
plt.yscale('log')
plt.legend(loc='best',fontsize=30)
plt.show()

In [72]:
model_chopper_lin = models.LinearModel()
guess_chopper_lin = model_chopper_lin.guess(np.log(data_chopper[1]),x=np.log(data_chopper[0]))
out_chopper_lin = model_chopper_lin.fit(np.log(data_chopper[1]),
                                        guess,
                                        x=np.log(data_chopper[0]),
                                        weights=1/((np.log(data_chopper[1]+data_chopper[3])+np.log(data_chopper[1]-data_chopper[3]))/2))

In [73]:
out_chopper_lin.plot()
plt.show()

In [74]:
print(out_chopper_lin.fit_report())

[[Model]]
    Model(linear)
[[Fit Statistics]]
    # function evals   = 7
    # data points      = 11
    # variables        = 2
    chi-square         = 0.001
    reduced chi-square = 0.000
[[Variables]]
    slope:      -1.02941998 +/- 0.027973 (2.72%) (init= 0.4495992)
    intercept:   8.89078789 +/- 0.099787 (1.12%) (init= 9.907773)
[[Correlations]] (unreported correlations are <  0.100)
    C(slope, intercept)          = -0.990 


# Laserinterferogramm

## Eichung


In [93]:
data_laser_eichung = np.genfromtxt('eichung_laser.dat',delimiter='\t').T

In [95]:
plt.plot(data_laser_eichung[1],data_laser_eichung[0],'. ')
plt.show()

## Interferogramm

In [132]:
data_laser = np.genfromtxt('laser.txt',delimiter='\t').T

In [133]:
plt.plot(data_laser[0],data_laser[1])
plt.xlim([0,30400])
plt.show()

# Weißlichtpunkt

## Eichung

In [124]:
data_weisslicht_eichung = np.genfromtxt('eichung_weisspunkt.dat',delimiter="\t").T

In [126]:
plt.plot(data_weisslicht[0],data_weisslicht[1])
plt.show()

In [129]:
model = models.LinearModel()
guesse = model.guess(data_weisslicht_eichung[1],x=data_weisslicht_eichung[0])
out = model.fit(data_weisslicht_eichung[1],guess,x=data_weisslicht_eichung[0])

In [130]:
out.plot_fit()
plt.show()

In [131]:
print(out.fit_report())

[[Model]]
    Model(linear)
[[Fit Statistics]]
    # function evals   = 7
    # data points      = 12
    # variables        = 2
    chi-square         = 0.002
    reduced chi-square = 0.000
[[Variables]]
    slope:       5.3328e-05 +/- 2.11e-06 (3.95%) (init= 1e-07)
    intercept:   4.99608815 +/- 0.030543 (0.61%) (init= 2)
[[Correlations]] (unreported correlations are <  0.100)
    C(slope, intercept)          = -0.992 


## Interfreogramm

In [134]:
data_weisslicht_eichung = np.genfromtxt('eichung_weisspunkt.dat',delimiter="\t").T

In [162]:
plt.plot(data_weisslicht[0],data_weisslicht[1],label='Messdaten')
plt.show()

# Schmalbandfilter

## Eichung

## Interferogramm

In [143]:
data_schmalband = np.genfromtxt('filter_4,7.txt',delimiter='\t').T

In [163]:
plt.plot(data_schmalband[0][450:1990],data_schmalband[1][450:1990],label='Messdaten')
plt.show()